<a href="https://colab.research.google.com/github/sledhead/simple_python/blob/main/webpage_scrapv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
#Will display upload box to user to allow a file to upload and encrypted.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install xmltodict

In [ ]:
import xmltodict

In [ ]:
import time

Section below will gather xml file for each year of the senate vote....
2017 to current

In [ ]:
!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_115_1.xml
!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_115_2.xml

!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_116_1.xml
!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_116_2.xml

!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_117_1.xml
!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_117_2.xml

!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_118_1.xml
!wget https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_118_2.xml

--2024-05-17 07:24:42--  https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_115_1.xml
Resolving www.senate.gov (www.senate.gov)... 104.96.219.104, 2600:1408:ec00:108c::1e3, 2600:1408:ec00:108e::1e3
Connecting to www.senate.gov (www.senate.gov)|104.96.219.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/xml]
Saving to: ‘vote_menu_115_1.xml’

vote_menu_115_1.xml     [ <=>                ] 149.40K  --.-KB/s    in 0.005s  

2024-05-17 07:24:42 (29.3 MB/s) - ‘vote_menu_115_1.xml’ saved [152990]

--2024-05-17 07:24:42--  https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_115_2.xml
Resolving www.senate.gov (www.senate.gov)... 104.96.219.104, 2600:1408:ec00:108c::1e3, 2600:1408:ec00:108e::1e3
Connecting to www.senate.gov (www.senate.gov)|104.96.219.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/xml]
Saving to: ‘vote_menu_115_2.xml’

vote_menu_115_2.xml     [ <=>          

Next Section below will open xml and then use this file to build the list of file links for gather senator votes

In [ ]:

full_http_link_file_path = []
vote_menu_file_lst = ['vote_menu_115_1.xml','vote_menu_115_2.xml','vote_menu_116_1.xml','vote_menu_116_2.xml','vote_menu_117_1.xml','vote_menu_117_2.xml','vote_menu_118_1.xml','vote_menu_118_2.xml']

for each_year_call_lst in vote_menu_file_lst:

  xml_file = open(each_year_call_lst, 'r')
  xml_data_str = xml_file.read()

  xml_to_dict_obj = xmltodict.parse(xml_data_str)

  congress_num = xml_to_dict_obj['vote_summary']['congress']
  congress_sec_num = xml_to_dict_obj['vote_summary']['session']

  congress_lst_vote = xml_to_dict_obj['vote_summary']['votes']['vote']



  base_link_path = 'https://www.senate.gov/legislative/LIS/roll_call_votes/'
  full_base_link_path = base_link_path + 'vote' + str(congress_num) + str(congress_sec_num) + '/vote_' + str(congress_num) + '_' + str(congress_sec_num) + '_'
  print(full_base_link_path)

  for each_vote_section in congress_lst_vote:
    single_vote_num = each_vote_section['vote_number']
    #build full link path for getting xml data....
    #https://www.senate.gov/legislative/LIS/roll_call_votes/vote1172/vote_117_2_00420.htm

    single_file_link_path = full_base_link_path + str(single_vote_num) + '.xml'
    full_http_link_file_path.append(single_file_link_path)


https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1152/vote_115_2_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1161/vote_116_1_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1162/vote_116_2_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1171/vote_117_1_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1172/vote_117_2_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1181/vote_118_1_
https://www.senate.gov/legislative/LIS/roll_call_votes/vote1182/vote_118_2_


# Test Section


In [ ]:
print(len(full_http_link_file_path))

2791


In [ ]:
for single_file_path in full_http_link_file_path:
  print(single_file_path)



# Build Senator Dict

Section below will loop through list of xml files and build a dict of all the votes within the time period.  Just selected senators will be recorded.

In [ ]:
missing_links = []
senator_history_lst = []

#full_http_link_file_path = ['https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_00271.xml', 'https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_00262.xml']
select_only_senators = ['Daines', 'Tester', 'Cruz', 'Schumer','Warren','Cotton','Sanders', 'Feinstein','Paul','Hirono']

for each_single_file_link in full_http_link_file_path:
  #gather the data from each single file link path
  r = requests.get(each_single_file_link)
  #print(r.status_code)
  #print(r.text)

  #make sure we got a file....
  if(r.status_code == 200):
    vote_call_details = xmltodict.parse(r.text)

    congress_id_num = vote_call_details['roll_call_vote']['congress']
    congress_session_num = vote_call_details['roll_call_vote']['session']
    vote_call_num = vote_call_details['roll_call_vote']['vote_number']
    vote_call_issue = vote_call_details['roll_call_vote']['vote_title']

    #setup loop for what each senator voted...
    senator_member_lst = vote_call_details['roll_call_vote']['members']['member']
    select_senator_vote_dict = {dict['last_name']:dict['vote_cast'] for dict in senator_member_lst if (dict['last_name'] in select_only_senators) == True }
    print('Currently working on: ', vote_call_num)

    new_row_dict = {}
    new_row_dict['congress'] = congress_id_num
    new_row_dict['session'] = congress_session_num
    new_row_dict['vote_number'] = vote_call_num
    new_row_dict['vote_title'] = vote_call_issue

    for each_sen_record in select_only_senators:
      #print(each_sen_mem['last_name'])

      new_row_dict[each_sen_record] = select_senator_vote_dict[each_sen_record]

      #add to list for later examation
      #after building list, will add to dataframe.

    senator_history_lst.append(new_row_dict)

  else:
    #something went wrong...
    #keep track of this problem
    missing_links.append(full_file_link_path)

  #before moving to the next record, pause for some time....
  time.sleep(0.25)

# Test Section

In [ ]:
#will use this section to test the above code.

select_only_senators = ['Daines', 'Tester', 'Cruz', 'Schumer', 'Baldwin', 'Bennet']

test_list = [{'member_full': 'Baldwin (D-WI)', 'last_name': 'Baldwin', 'first_name': 'Tammy', 'party': 'D', 'state': 'WI', 'vote_cast': 'Yea', 'lis_member_id': 'S354'}, {'member_full': 'Barrasso (R-WY)', 'last_name': 'Barrasso', 'first_name': 'John', 'party': 'R', 'state': 'WY', 'vote_cast': 'Not Voting', 'lis_member_id': 'S317'}, {'member_full': 'Bennet (D-CO)', 'last_name': 'Bennet', 'first_name': 'Michael', 'party': 'D', 'state': 'CO', 'vote_cast': 'Yea', 'lis_member_id': 'S330'}]
print(len(test_list))

select_senator_vote_dict = {dict['last_name']:dict['vote_cast'] for dict in test_list if (dict['last_name'] in select_only_senators) == True }

print(select_senator_vote_dict)


3
{'Baldwin': 'Yea', 'Bennet': 'Yea'}


In [ ]:
print(missing_links)

for each_vote_cnt in senator_history_lst:

  print(each_vote_cnt)

[]
{'congress': '115', 'session': '1', 'vote_number': '271', 'vote_title': 'Motion to Invoke Cloture Re: Steven Gill Bradbury to be General Counsel of the Department of Transportation', 'Daines': 'Yea', 'Tester': 'Nay', 'Cruz': 'Yea', 'Schumer': 'Nay'}
{'congress': '115', 'session': '1', 'vote_number': '262', 'vote_title': 'Confirmation John H. Gibson II, of Texas, to be Deputy Chief Management Officer of the Department of Defense', 'Daines': 'Yea', 'Tester': 'Yea', 'Cruz': 'Yea', 'Schumer': 'Yea'}


In [ ]:
print(missing_links)
print(len(senator_history_lst))
print(senator_history_lst[155])

[]
2791
{'congress': '115', 'session': '1', 'vote_number': '170', 'vote_title': 'Donnelly Motion to Commit H.R. 1628 with Instructions', 'Daines': 'Nay', 'Tester': 'Yea', 'Cruz': 'Nay', 'Schumer': 'Yea', 'Warren': 'Yea', 'Cotton': 'Nay', 'Sanders': 'Yea'}


# Senator Track Record

# Test Section